In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
color = sns.color_palette()
pd.options.mode.chained_assignment = None  # default='warn'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
df1 = pd.read_excel("2019_bronx.xlsx")
df2 = pd.read_excel("2019_brooklyn.xlsx")
df3 = pd.read_excel("2019_manhattan.xlsx")
df4 = pd.read_excel("2019_queens.xlsx")
df5 = pd.read_excel("2019_statenisland.xlsx")

In [5]:
df1['AREA'] = 'bronx'
df2['AREA'] = 'brooklyn'
df3['AREA'] = 'manhattan'
df4['AREA'] = 'queens'
df5['AREA'] = 'statenisland'

In [6]:
frames = [df1, df2, df3, df4, df5]
df = pd.concat(frames)

In [7]:
df.columns = ['BOROUGH',
                  'NEIGHBORHOOD',
                  'BUILDING CLASS CATEGORY',
                  'TAX CLASS AS OF FINAL ROLL 18/19',
                  'BLOCK',
                  'LOT',
                  'EASE-MENT',
                  'BUILDING CLASS AS OF FINAL ROLL 18/19',
                  'ADDRESS',
                  'APARTMENT NUMBER',
                  'ZIP CODE',
                  'RESIDENTIAL UNITS',
                  'COMMERCIAL UNITS',
                  'TOTAL UNITS',
                  'LAND SQUARE FEET',
                  'GROSS SQUARE FEET',
                  'YEAR BUILT',
                  'TAX CLASS AT TIME OF SALE',
                  'BUILDING CLASS AT TIME OF SALE',
                  'SALE PRICE',
                  'SALE DATE',
                  'AREA']

In [8]:
df['FINAL_ADDRESS'] = df['ADDRESS'].apply(lambda x: x + " NYC")

In [9]:
df = df.drop(['EASE-MENT', 'APARTMENT NUMBER'], axis=1)

In [10]:
df = df[df['SALE PRICE'] > 300000]
df = df[df['SALE PRICE'] < 2e6]

df = df[df['RESIDENTIAL UNITS'] < 10.0]

df = df[df['COMMERCIAL UNITS'] < 5.0]

df = df[df['YEAR BUILT'] > 1900]
df = df[df['YEAR BUILT'] <= 2020]

df = df[df['GROSS SQUARE FEET'] < 4600]
df = df[df['GROSS SQUARE FEET'] > 50]

df = df[df['LAND SQUARE FEET'] > 200]
df = df[df['LAND SQUARE FEET'] < 5000]

df = df[df['ZIP CODE'] > 10000]
df = df[df['ZIP CODE'] < 12000]

df = df[df['TOTAL UNITS'] < 5]
df = df[df['TOTAL UNITS'] > 0]

df = df[df['LOT'] < 150]

In [11]:
df = df.dropna(how='any', axis=0)

In [12]:
print(df.shape[0])
print(df.shape[1])

17693
21


In [13]:
# split the data into train and test set
df_use, df_not_use = train_test_split(df, test_size=0.5, random_state=42, shuffle=True)

In [14]:
print(df_use.shape[0])
print(df_use.shape[1])

8846
21


In [15]:
df_use

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AS OF FINAL ROLL 18/19,BLOCK,LOT,BUILDING CLASS AS OF FINAL ROLL 18/19,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,AREA,FINAL_ADDRESS
2234,2,HIGHBRIDGE/MORRIS HEIGHTS,03 THREE FAMILY DWELLINGS,1,2872,105,C0,1432 SHAKESPEARE AVENUE,10452.0,3.0,0.0,3.0,3645.0,4422.0,1925.0,1,C0,786500,2019-06-19,bronx,1432 SHAKESPEARE AVENUE NYC
576,4,ASTORIA,02 TWO FAMILY DWELLINGS,1,874,63,B2,21-13 24TH ROAD,11102.0,2.0,0.0,2.0,1360.0,800.0,1940.0,1,B2,1225000,2019-09-26,queens,21-13 24TH ROAD NYC
17382,4,NEPONSIT,01 ONE FAMILY DWELLINGS,1,16314,15,A1,215 BEACH 149TH STREET,11694.0,1.0,0.0,1.0,4500.0,2200.0,1960.0,1,A1,999000,2019-11-07,queens,215 BEACH 149TH STREET NYC
15226,4,LAURELTON,01 ONE FAMILY DWELLINGS,1,12938,60,A1,131-40 229TH STREET,11413.0,1.0,0.0,1.0,3200.0,1748.0,1940.0,1,A1,640000,2019-01-29,queens,131-40 229TH STREET NYC
18351,3,OCEAN PARKWAY-SOUTH,01 ONE FAMILY DWELLINGS,1,7089,54,A1,705 AVENUE T,11223.0,1.0,0.0,1.0,2100.0,1630.0,1930.0,1,A1,1325000,2019-05-14,brooklyn,705 AVENUE T NYC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17043,4,MIDDLE VILLAGE,01 ONE FAMILY DWELLINGS,1,3039,33,A5,65-55 77TH PLACE,11379.0,1.0,0.0,1.0,1800.0,1224.0,1935.0,1,A5,780000,2019-11-20,queens,65-55 77TH PLACE NYC
18726,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,11158,27,A1,103-06 223RD STREET,11429.0,1.0,0.0,1.0,2850.0,1782.0,1925.0,1,A1,510000,2019-09-27,queens,103-06 223RD STREET NYC
13291,3,FLATBUSH-NORTH,02 TWO FAMILY DWELLINGS,1,4843,11,B1,84 EAST 39TH STREET,11203.0,2.0,0.0,2.0,1842.0,1980.0,1950.0,1,B1,690000,2019-01-15,brooklyn,84 EAST 39TH STREET NYC
2815,2,MELROSE/CONCOURSE,03 THREE FAMILY DWELLINGS,1,2405,108,C0,816 COURTLANDT AVENUE,10451.0,3.0,0.0,3.0,2116.0,3174.0,2002.0,1,C0,950000,2019-12-30,bronx,816 COURTLANDT AVENUE NYC


In [16]:
df_use.to_csv('data_after_pre_processing.csv', index=False)